In [1]:
import subprocess

# function to execute shell commands (this is for convenience)
def runshell(CMD):
    global ERRORS
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:  # new add: handle exceptions 
        return -1, str(e)

### Write Permission to Home Directory Test

In [55]:

# Test to check write permissions to home directory
def WritePermissionTest():

    TEST = "WRITE PERMISSION TO HOME DIRECTORY"
    CMD = f"touch {os.path.expanduser('~')}/test_write_permissions.tmp && echo 'Write Permission: Yes' && rm {os.path.expanduser('~')}/test_write_permissions.tmp || echo 'Write Permission: No'"

    e, output = runshell(CMD)

    if e == 0:
        PASSES.append("Write Permission to Home Directory test")

    else:
        ERRORS.append(output)


### Environmental Variables test

In [53]:
#Verify environment variables are correct

def check_environment_test():
    
    NB_UID = int(os.environ['NB_UID'])
    NB_GID = int(os.environ['NB_GID'])
    NB_GROUP = os.environ['NB_GROUP']
    NB_USER = os.environ['NB_USER']

    XDG_CACHE_HOME = f"/home/{NB_USER}/.cache/"

    UID_LOWER_BOUND = 1000  
    UID_UPPER_BOUND = 1000810001 

    GID_LOWER_BOUND = 0  
    GID_UPPER_BOUND = 10000 

    EXPECTED_NB_GROUP = 'root'

    err = []    
    
    if not (UID_LOWER_BOUND <= NB_UID <= UID_UPPER_BOUND):
         err.append(f"NB_UID {NB_UID} is not within the acceptable range: {UID_LOWER_BOUND}-{UID_UPPER_BOUND}.")
         err.append("Acceptable ranges can be modified within: " + os.getcwd()) 

    if not (GID_LOWER_BOUND <= NB_GID <= GID_UPPER_BOUND):
         err.append(f"NB_GID {NB_GID} is not within the acceptable range: {GID_LOWER_BOUND}-{GID_UPPER_BOUND}.")
         err.append("Acceptable ranges can be modified within: " + os.getcwd())              
            
    if  NB_GROUP != EXPECTED_NB_GROUP:
         err.append("NB_GROUP does not match " + "'" + EXPECTED_NB_GROUP + "'")

    if NB_USER != 'jovyan':
         err.append("NB_USER does not match 'jovyan'.")

    if XDG_CACHE_HOME != '/home/jovyan/.cache/':
         err.append("XDD_CACHE_HOME does not match expected path: /home/jovyan/.cache/")

    s = '\n '.join(err)
    if not s: # no errors
        PASSES.append("Environmental Variables test")
    else:
        ERRORS.append(f"Environmental Variables test:\n {s}")
       

### ASLR Test

In [18]:
# run gdb 100 times on date binary stopping at first instruction and grep ld. If ld is found at same address (following 
# command has a value of 1 then ASLR is properly disabled.
# We want to disable ASLR so that students can use GDB properly in their environments.
def ASLRtest():
    e, count = runshell("echo $(for ((i=0;i<100; i++)); do gdb -ex starti -ex quit -q --batch /usr/bin/date 2>/dev/null | grep ld; done | tee out | uniq | wc -l)")
    if count == 1: 
        PASSES.append("ASLR test")
    else:
        e2, output2 = runshell("echo $(cat /proc/sys/kernel/randomize_va_space)")
        ERRORS.append(f"ASLR test ERROR: ASLR status is enabled with status {output2[0]} (0 is disabled, 1 is partial and 2 is full)")

### Network Test

In [20]:
# Curl test to check internet connectivity
def NetworkTest():
    e, output = runshell("curl google.com")
    if e == 0:
        PASSES.append("Network test")
    else:
        ERRORS.append("Network test ERROR: " + output)

### Pip-Conda Test

In [11]:
def PipCondaTest():
    TEST = "PIP PACKAGE INSTALLATION"
    CMD = "pip install --user pytest"
    e, output = runshell(CMD)

    if e == 0:
        PASSES.append("Pip-Conda test")
    else:
        ERRORS.append("Pip-Conda test: " + output)

### Git and SSH Test

In [54]:
def GitSSHTest():

    e, output = runshell("readlink -f ~/.gitconfig")

    if e == 0:
        PASSES.append("Git config test")
    else:
        ERRORS.append("Git config test:" + output)

    e, output = runshell("readlink -f /etc/ssh/ssh_config")

    if e == 0:
        PASSES.append("ssh config test")

    else:
        ERRORS.append("ssh config test:" + output)


### Conda Directory Test

In [51]:
"""Check if a directory is readable and writable."""
def check_permissions():
    
    #Identify conda directories

    conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
    conda_env_dir = os.environ.get('CONDA_PREFIX', '')

    unaccessible_dirs = 0    
    
    for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), ('Conda Environment Directory', conda_env_dir)]:
        try:
            readable = os.access(dir_path, os.R_OK)
            writable = os.access(dir_path, os.W_OK)
            if not (readable and writable):
                unaccessible_dirs += 1
                ERRORS.append(f"Conda Directory test ERROR: {dir_name} at '{dir_path}' is not readable/writable")
        except Exception as e:
            ERRORS.append(f"Conda Directory test ERROR: Error checking permissions for {dir_name} at {dir_path}: {e}")
            unaccessible_dirs += 1

    if unaccessible_dirs == 0:
        PASSES.append("Conda directory r/w test passed")


### Running the tests

In [ ]:
import os

# global var to keep track of test results we have
ERRORS = []
PASSES = []


#Run the tests

WritePermissionTest()
check_environment_test()
ASLRtest()
NetworkTest()
GitSSHTest()
check_permissions()

#Print the results
if len(ERRORS) > 0:
    print("FAILED TESTS:")
    print('-' + '\n-'.join(ERRORS))
    print('\n')
    print("PASSED TESTS:")
    print('-' + '\n-'.join(PASSES))
else:
    print("ALL TESTS PASS")
    print('-' + '\n-'.join(PASSES))